# DDPM Inverse Markov Chain (Ho et al., 2020)

## 1. Notation

- 設 $\beta_t \in (0,1)$，$\alpha_t := 1-\beta_t$，$\bar\alpha_t := \prod_{s=1}^t \alpha_s$。  
- 前向（加噪）一步：

  $$
  q(x_t \mid x_{t-1}) = \mathcal{N}\!\bigl(x_t; \sqrt{\alpha_t}\,x_{t-1},\,\beta_t I\bigr)
  $$

- 對任意 $t$ 的閉式抽樣：

  $$
  q(x_t \mid x_0) = \mathcal{N}\!\bigl(x_t; \sqrt{\bar\alpha_t}\,x_0,\,(1-\bar\alpha_t)I\bigr)
  $$


## 2. 真實的「Bayes 逆向」後驗（理想）

利用高斯線性模型的共軛性，可得對任意 $t \ge 1$：

$$
q(x_{t-1}\mid x_t, x_0) = \mathcal{N}\!\bigl(x_{t-1};\,\tilde\mu_t(x_t,x_0),\,\tilde\beta_t I\bigr)
$$

其中：

$$
\tilde\beta_t = \frac{1-\bar\alpha_{t-1}}{1-\bar\alpha_t}\,\beta_t, \qquad
\tilde\mu_t(x_t,x_0) = \frac{\sqrt{\bar\alpha_{t-1}}\,\beta_t}{1-\bar\alpha_t}\,x_0 + \frac{\sqrt{\alpha_t}\,(1-\bar\alpha_{t-1})}{1-\bar\alpha_t}\,x_t
$$

> 注意：$\tilde\mu_t$ 依賴未知的 $x_0$，所以不能直接生成。

## 3. 論文中的「inverse（反向）鏈」參數化

以可學模型 $p_\theta$ 近似 $q$ 的反向轉移：

$$
p_\theta(x_{t-1}\mid x_t) = \mathcal{N}\!\bigl(x_{t-1};\,\mu_\theta(x_t,t),\,\Sigma_\theta(x_t,t)\bigr), \quad t = T,\dots,1
$$

### (a) 以「預測噪音」$\epsilon_\theta$ 的參數化

Forward relation:

$$
x_t = \sqrt{\bar\alpha_t}\,x_0 + \sqrt{1-\bar\alpha_t}\,\epsilon, \quad \epsilon \sim \mathcal{N}(0,I)
$$

訓練模型 $\epsilon_\theta(x_t, t) \approx \epsilon$，並定義：

$$
\hat{x}_0(x_t,t) = \frac{1}{\sqrt{\bar\alpha_t}}\Bigl(x_t - \sqrt{1-\bar\alpha_t}\,\epsilon_\theta(x_t,t)\Bigr)
$$

帶回 $\tilde\mu_t$：

$$
\mu_\theta(x_t,t) = \tilde\mu_t(x_t, \hat{x}_0) = \frac{1}{\sqrt{\alpha_t}}\!\left(x_t - \frac{\beta_t}{\sqrt{1-\bar\alpha_t}}\epsilon_\theta(x_t,t)\right)
$$

方差部分採固定形式：

$$
\Sigma_\theta(x_t,t) = \tilde\beta_t I
$$


## 4. 反向（生成）演算法 —— Inverse Chain

1. 取 $x_T \sim \mathcal{N}(0,I)$  
2. 對 $t = T, T-1, \dots, 2$：

   $$
   \mu_\theta = \frac{1}{\sqrt{\alpha_t}}\!\left(x_t - \frac{\beta_t}{\sqrt{1-\bar\alpha_t}}\epsilon_\theta(x_t,t)\right)
   $$

   $$
   \sigma_t^2 = \tilde\beta_t
   $$

   $$
   x_{t-1} \sim \mathcal{N}\!\bigl(\mu_\theta,\,\sigma_t^2 I\bigr)
   $$

3. 在 $t=1$ 時，通常使用：
   $$
   x_0 = \mu_\theta(x_1, 1)
   $$


## 5. 兩個常見等價寫法

- **$\hat{x}_0$-prediction：** 先預測 $\hat{x}_0$，再代入 $\tilde\mu_t(x_t, \hat{x}_0)$。
- **$v$-prediction：** 另一種數值穩定參數化，與 $\epsilon$ 版本等價。